In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, fixed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Note: This notebook assumes the functions
# load_ss433_params, ss433_phases, and _plot_jet_trajectories_on_ax
# have been defined (e.g., by running them in a cell above).
# You may need to copy Cells 1, 2, and parts of 4 into this new notebook.

# --- 1. Load Data and Parameters ---
# Ensure the CSV file is accessible
try:
    # Make sure CSV_INPUT_PATH is defined
    df_interactive = pd.read_csv(CSV_INPUT_PATH)
    obs_id_list = sorted(df_interactive['obs_id'].unique())
    print("Data loaded for interactive plot.")
except NameError:
    print("Please make sure CSV_INPUT_PATH is defined and the data loading cell has been run.")
    df_interactive = pd.DataFrame()
    obs_id_list = []
except FileNotFoundError:
    print(f"Interactive data file not found at {CSV_INPUT_PATH}")
    df_interactive = pd.DataFrame()
    obs_id_list = []


# Load the full ephemeris model for the overlay
params_full = load_ss433_params(source='full_russian')

# --- 2. Create the Interactive Plotting Function ---
def interactive_beta_plot(obs_id, base_beta):
    """
    Plots an observation and overlays the full model, 
    with beta controlled by a slider.
    """
    if df_interactive.empty:
        print("DataFrame is empty. Cannot generate plot.")
        return

    # --- Get observation data ---
    group = df_interactive[df_interactive['obs_id'] == obs_id]
    if group.empty:
        print(f"No data found for Obs ID: {obs_id}")
        return
        
    try:
        blue_data = group[group['component'] == 'g2'].iloc[0]
        red_comp_name = sorted(group['component'].unique())[-1]
        red_data = group[group['component'] == red_comp_name].iloc[0]
        mjd_obs = blue_data['mjd']
    except (IndexError, StopIteration):
        print(f"Could not find valid blue/red pair for Obs ID: {obs_id}")
        return

    # --- Create the plot ---
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': 'polar'})
    
    # --- Plot observed data points with error bars ---
    ax.errorbar(np.deg2rad(blue_data['PA']), blue_data['radius'], 
                yerr=[[abs(blue_data['radius_minus_err'])], [abs(blue_data['radius_plus_err'])]], 
                xerr=[[np.deg2rad(abs(blue_data['PA_err_minus']))], [np.deg2rad(abs(blue_data['PA_err_plus']))]], 
                fmt='o', color='blue', ecolor='gray', capsize=4, zorder=5, label="Observed Blue Jet")
    
    ax.errorbar(np.deg2rad(red_data['PA']), red_data['radius'], 
                yerr=[[abs(red_data['radius_minus_err'])], [abs(red_data['radius_plus_err'])]], 
                xerr=[[np.deg2rad(abs(red_data['PA_err_minus']))], [np.deg2rad(abs(red_data['PA_err_plus']))]], 
                fmt='o', color='red', ecolor='gray', capsize=4, zorder=5, label="Observed Red Jet")

    # --- Plot the full model with the beta from the slider ---
    temp_params = params_full.copy()
    temp_params['beta'] = base_beta # Use the slider value
    
    # Use a limited time range for the colorbar (e.g., one precession period)
    mappable = plt.cm.ScalarMappable(cmap=plt.cm.rainbow, norm=mcolors.Normalize(vmin=1.0, vmax=162.5))
    # Plot the trajectories
    _plot_jet_trajectories_on_ax(ax, mjd_obs, temp_params, mappable, r_max=2.0)
    
    # --- Set plot aesthetics ---
    max_rad_obs = max(blue_data['radius_plus_err'], red_data['radius_plus_err'])
    ax.set_rmin(0)
    ax.set_rmax(max(1.4, max_rad_obs * 1.2)) # Set rmax dynamically
    ax.set_title(f"Model Overlay for Obs ID: {obs_id}\nBase Beta = {base_beta:.4f}", pad=20)
    ax.legend()
    plt.show()

# --- 3. Create and Display the Widgets ---
if not df_interactive.empty:
    # continuous_update=False is key to preventing choppiness
    beta_slider = widgets.FloatSlider(
        min=0.2, 
        max=0.32, 
        step=0.0005, 
        value=params_full['beta'], 
        description='Beta:',
        continuous_update=False, 
        readout_format='.4f',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    obs_id_dropdown = widgets.Dropdown(
        options=obs_id_list,
        description='Obs ID:',
        style={'description_width': 'initial'}
    )

    # Link the widgets to the plotting function
    interact(interactive_beta_plot, obs_id=obs_id_dropdown, base_beta=beta_slider);
else:
    print("Skipping interactive widget creation as no data was loaded.")